In [ ]:
class Credit:
    def __init__(
        self,
        id_ekspozycji: str,
        stage: int,
        pd: list,              # miesięczne PD
        lgd: list,             # miesięczne LGD dla części niezabezpieczonej
        ead_total: list,       # całkowita EAD (zabezpieczona + niezabezpieczona)
        ead_collateral: list,  # część zabezpieczona EAD
        rr_collateral: float,  # recovery rate zabezpieczenia (0–1)
        life_months: int,      # liczba miesięcy do końca trwania ekspozycji
        esp: float # roczna efektywna stopa oprocentowania 
    ):
        self.id = id_ekspozycji
        self.stage = stage
        self.pd = pd
        self.lgd = lgd
        self.ead_total = ead_total
        self.ead_collateral = ead_collateral
        self.rr_collateral = rr_collateral
        self.life_months = life_months
        self.esp = esp
        self.odpis = 0.0

        self._validate_inputs()

    def _validate_inputs(self):
        expected_len = self.life_months if self.stage == 1 else min(self.life_months, 12)
        for name, arr in [
            ("PD", self.pd),
            ("LGD", self.lgd),
            ("EAD total", self.ead_total),
            ("EAD zabezpieczona", self.ead_collateral)
        ]:
            if len(arr) < expected_len:
                raise ValueError(f"{name} musi mieć co najmniej {expected_len} miesięcy.")
        if not (0 <= self.rr_collateral <= 1):
            raise ValueError("RR_collateral musi być w przedziale [0, 1].")
        for t in range(expected_len):
            if self.ead_collateral[t] > self.ead_total[t]:
                raise ValueError(f"W miesiącu {t} zabezpieczenie przekracza EAD.")

    def ecl_calc(self):
        if self.stage == 1:
            self.odpis = self._calculate_stage1()
        elif self.stage == 2:
            self.odpis = self._calculate_stage2()
        elif self.stage == 3:
            self.odpis = self._calculate_stage3()
        else:
            raise ValueError("Stage musi być 1, 2 lub 3.")

    def _loss_given_default_amount(self, t: int) -> float:
        """
        Zwraca kwotową stratę warunkową przy default (LGD x EAD), uwzględniając zabezpieczenie.
        """
        ead_zabezp = self.ead_collateral[t]
        ead_niezabezp = self.ead_total[t] - ead_zabezp

        loss_zabezp = ead_zabezp * (1 - self.rr_collateral)
        loss_niezabezp = ead_niezabezp * self.lgd[t]

        return loss_zabezp + loss_niezabezp

    def _calculate_stage1(self) -> float:
        n_months = min(12, self.life_months)
        ecl = 0.0
        for t in range(n_months):
            discount = 1 / ((1 + self.esp) ** ((t + 1)/12))
            lgd_amt = self._loss_given_default_amount(t)
            ecl += self.pd[t] * lgd_amt * discount
        return ecl

    def _calculate_stage2(self) -> float:
        ecl = 0.0
        for t in range(self.life_months):
            discount = 1 / ((1 + self.esp) ** ((t + 1)/12))
            lgd_amt = self._loss_given_default_amount(t)
            ecl += self.pd[t] * lgd_amt * discount
        return ecl

    def _calculate_stage3(self) -> float:
        return self._loss_given_default_amount(0)


In [20]:
import unittest

class TestCredit(unittest.TestCase):
    def setUp(self):
        # Dane przykładowe (życiowe)
        self.id_eksp = "KRED123"
        self.stage = 1
        self.life_months = 3
        self.esp = 0.01  # 1% miesięczna stopa dyskontowa
        self.rr_collateral = 0.4

        self.pd = [0.01, 0.02, 0.03]  # PD miesięczne
        self.lgd = [0.5, 0.5, 0.5]    # LGD miesięczne (część niezabezpieczona)
        self.ead_total = [1000, 900, 800]
        self.ead_collateral = [200, 180, 160]  # zabezpieczona część

    def test_init_validation(self):
        # Poprawna inicjalizacja nie powinna rzucać błędów
        c = Credit(self.id_eksp, self.stage, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)
        self.assertEqual(c.id, self.id_eksp)

        # Zabezpieczenie większe niż EAD
        with self.assertRaises(ValueError):
            Credit(self.id_eksp, self.stage, self.pd, self.lgd, self.ead_total,
                   [1100, 3000, 160], self.rr_collateral, self.life_months, self.esp)

        # RR poza przedziałem
        with self.assertRaises(ValueError):
            Credit(self.id_eksp, self.stage, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, 1.5, self.life_months, self.esp)

        # Za krótka lista pd
        with self.assertRaises(ValueError):
            Credit(self.id_eksp, self.stage, [0.01], self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)

    def test_loss_given_default_amount(self):
        c = Credit(self.id_eksp, self.stage, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)

        # Dla miesiąca 0
        loss0 = c._loss_given_default_amount(0)
        # ead zabezp = 200, niezabezp = 800
        # loss_zabezp = 200 * (1 - 0.4) = 120
        # loss_niezabezp = 800 * 0.5 = 400
        # suma = 520
        self.assertAlmostEqual(loss0, 520)

        # Dla miesiąca 1
        loss1 = c._loss_given_default_amount(1)
        # ead zabezp = 180, niezabezp = 720
        # loss_zabezp = 180 * 0.6 = 108
        # loss_niezabezp = 720 * 0.5 = 360
        # suma = 468
        self.assertAlmostEqual(loss1, 468)

    def test_ecl_calc_stage1(self):
        c = Credit(self.id_eksp, 1, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)
        c.ecl_calc()
        # ręczne liczenie (tylko 3 miesiące bo life_months=3<12)
        # ECL = sum_{t=0}^{2} PD_t * LGD_EAD_t * discount_t
        # t=0: 0.01 * 520 * 1/(1+0.01)^1 = 0.01*520/1.01 ≈ 5.1485
        # t=1: 0.02 * 468 * 1/(1+0.01)^2 = 0.02*468/1.0201 ≈ 9.1753
        # t=2: 0.03 * loss(2) * discount(3)
        # loss(2) = ead_collateral=160 * 0.6 + (800-160)*0.5 = 160*0.6 + 640*0.5 = 96 + 320 = 416
        # discount(3) = 1/(1.01^3) ≈ 0.9706
        # ECL_t2 = 0.03 * 416 * 0.9706 ≈ 12.116
        # suma ≈ 5.1485 + 9.1753 + 12.116 = 26.44
        expected_ecl = 5.1485 + 9.1753 + 12.116
        self.assertAlmostEqual(c.odpis, expected_ecl, places=2)

    def test_ecl_calc_stage2(self):
        c = Credit(self.id_eksp, 2, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)
        c.ecl_calc()
        # stage 2 = liczymy dla całej długości życia (3 miesiące)
        # powinna być równa stage1, bo life_months=3 < 12
        expected_ecl = c._calculate_stage2()
        self.assertAlmostEqual(c.odpis, expected_ecl)

    def test_ecl_calc_stage3(self):
        c = Credit(self.id_eksp, 3, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)
        c.ecl_calc()
        # stage 3 = loss given default dla pierwszego miesiąca (t=0)
        expected_loss = c._loss_given_default_amount(0)
        self.assertAlmostEqual(c.odpis, expected_loss)

    def test_invalid_stage(self):
        c = Credit(self.id_eksp, 4, self.pd, self.lgd, self.ead_total,
                   self.ead_collateral, self.rr_collateral, self.life_months, self.esp)
        with self.assertRaises(ValueError):
            c.ecl_calc()

In [21]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_ecl_calc_stage1 (__main__.TestCredit.test_ecl_calc_stage1) ... ok
test_ecl_calc_stage2 (__main__.TestCredit.test_ecl_calc_stage2) ... ok
test_ecl_calc_stage3 (__main__.TestCredit.test_ecl_calc_stage3) ... ok
test_init_validation (__main__.TestCredit.test_init_validation) ... ok
test_invalid_stage (__main__.TestCredit.test_invalid_stage) ... ok
test_loss_given_default_amount (__main__.TestCredit.test_loss_given_default_amount) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.004s

OK
